# AMPL Model Colaboratory Template
[![colab.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/Blova-TB/OptimisationUQAC) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Blova-TB/OptimisationUQAC/blob/Probleme-3/template/colab.ipynb)

Template by Filipe Brandão <<fdabrandao@gmail.com>>

In [12]:
# Install dependencies
%pip install -q amplpy pandas

In [13]:
# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://ampl.com/colab).


## Use `%%ampl_eval` to evaluate AMPL commands and declarations

In [14]:
%%ampl_eval

set POUDRE; # A ou B
set PRODUIT; # enceinte ou allaite
set NUTRIMENT; # Lipide ou Proteine

param prix {PRODUIT} >= 0;
param cout {POUDRE} >= 0;
param maxVenteProduit {PRODUIT} >= 0;
param minVenteProduit {PRODUIT} >= 0;
param disponibilitePoudre {POUDRE} >= 0;
param compositionMaxProduit {PRODUIT,NUTRIMENT} >= 0;
param compositionMinProduit {PRODUIT,NUTRIMENT} >= 0;
param compositionPoudre {POUDRE,NUTRIMENT} >= 0;

var achat {POUDRE,PRODUIT} >=0;
var vente {prod in PRODUIT} >= minVenteProduit[prod], <= maxVenteProduit[prod];

maximize Total_Cost: sum {prod in PRODUIT} vente[prod] * prix[prod] - sum {po in POUDRE,prod in PRODUIT} achat[po,prod] * cout[po];

subject to Max_Composition_Nutriment {prod in PRODUIT, nut in NUTRIMENT}:
    vente[prod] * compositionMaxProduit[prod,nut] >= sum {po in POUDRE} achat[po,prod] * compositionPoudre[po,nut];

subject to Min_Composition_Nutriment {prod in PRODUIT, nut in NUTRIMENT}:
    vente[prod] * compositionMinProduit[prod,nut] <= sum {po in POUDRE} achat[po,prod] * compositionPoudre[po,nut];

subject to Disponibilite_Poudre {po in POUDRE}:
    sum {prod in PRODUIT} achat[po,prod] <= disponibilitePoudre[po];

## Load data directly from Python data structures using [amplpy](https://amplpy.readthedocs.io/)

In [15]:
poudre = ["A","B"]
ampl.set["POUDRE"] = poudre

produit = ["enceinte","allaite"]
ampl.set["PRODUIT"] = produit

nut = ["Lipide","Proteine"]
ampl.set["NUTRIMENT"] = nut

ampl.param["prix"] = {
    "enceinte": 18,
    "allaite": 25
}

ampl.param["cout"] = {
    "A": 19,
    "B": 9
}

ampl.param["maxVenteProduit"] = {
    "enceinte": 20,
    "allaite": 30
}

ampl.param["minVenteProduit"] = {
    "enceinte": 15,
    "allaite": 22
}

ampl.param["disponibilitePoudre"] = {
    "A": 27,
    "B": 31
}

ampl.param["compositionMaxProduit"] = {
    ("enceinte","Proteine"): 20,
    ("enceinte","Lipide"): 40,
    ("allaite","Proteine"):20,
    ("allaite","Lipide"):45
}

ampl.param["compositionMinProduit"] = {
    ("enceinte","Proteine"): 12,
    ("enceinte","Lipide"): 30,
    ("allaite","Proteine"):12,
    ("allaite","Lipide"):20
}

ampl.param["compositionPoudre"] = {
    ("A","Proteine"):35,
    ("A","Lipide"):38,
    ("B","Proteine"):5,
    ("B","Lipide"):32
}

## Solve with HiGHS

In [16]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1"
ampl.solve()

HiGHS 1.11.0:   tech:outlev = 1

AMPL MP initial flat model has 6 variables (0 integer, 0 binary);
Objectives: 1 linear; 
Constraints:  10 linear;

AMPL MP did not modify the model.


Running HiGHS 1.11.0 (git hash: 364c83a51): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 10 rows; 6 cols; 28 nonzeros
Coefficient ranges:
  Matrix [1e+00, 4e+01]
  Cost   [9e+00, 2e+01]
  Bound  [2e+01, 3e+01]
  RHS    [3e+01, 3e+01]
Presolving model
10 rows, 6 cols, 28 nonzeros  0s
10 rows, 6 cols, 28 nonzeros  0s
Presolve : Reductions: rows 10(-0); columns 6(-0); elements 28(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          4     6.5451612903e+02 Pr: 0(0) 0s
Model status        : Optimal
Simplex   iterations: 4
Objective value     :  6.5451612903e+02
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00

## Retrieve solution as a pandas dataframe

In [17]:
ampl.var["achat"].to_pandas()

achat.val
index0 index1             
A      allaite    9.161290
       enceinte   5.032258
B      allaite    7.870968
       enceinte  12.774194

In [18]:
ampl.var["vente"].to_pandas()

,vente.val
allaite,30
enceinte,20
